In [28]:
import pandas as pd
import numpy as np
import requests as rq

In [12]:
iv = pd.read_csv('iv.csv')


In [22]:
#sort according to the column 'date'
iv = iv.sort_values(by='date')
iv.head()


,date,symbol,open,high,low,close
960,2021-03-24,BTC,84.80,95.94,80.52,95.04
959,2021-03-25,BTC,95.03,97.43,87.25,89.74
958,2021-03-26,BTC,89.74,89.74,80.22,84.52
957,2021-03-27,BTC,84.52,84.60,77.81,79.69
956,2021-03-28,BTC,79.69,81.28,78.20,79.07


In [52]:
import requests
import pandas as pd
from datetime import datetime as dt
from datetime import date, timedelta

def datetime_to_timestamp(datetime_obj): 
    """Converts a datetime object to a Unix timestamp in milliseconds."""
    return int(dt.timestamp(datetime_obj)*1000)

def timestamp_to_datetime(timestamp): 
    """Converts a Unix timestamp in milliseconds to a datetime object."""
    return dt.fromtimestamp(timestamp/1000)

def derivative_data(currency: str, kind: str, start_date: date, end_date: date, count: int = 10000) -> pd.DataFrame:
    """Returns derivative trade data for a specified currency and time range.

    Args:
        currency (str): The currency symbol, e.g. 'BTC'.
        kind (str): The type of derivative, either 'option' or 'future'.
        start_date (date): The start date of the time range (inclusive).
        end_date (date): The end date of the time range (inclusive).
        count (int, optional): The maximum number of trades to retrieve per request. Defaults to 10000.

    Returns:
        pandas.DataFrame: A dataframe of derivative trade data for the specified currency and time range.
    """

    # Validate input arguments
    assert isinstance(currency, str), "currency must be a string"
    assert isinstance(start_date, date), "start_date must be a date object"
    assert isinstance(end_date, date), "end_date must be a date object"
    assert start_date <= end_date, "start_date must be before or equal to end_date"

    derivative_list = []
    params = {
        "currency": currency, 
        "kind": kind,
        "count": count,
        "include_old": True,
        "start_timestamp": datetime_to_timestamp(dt.combine(start_date, dt.min.time())),
        "end_timestamp": datetime_to_timestamp(dt.combine(end_date, dt.max.time()))
    }

    url = 'https://history.deribit.com/api/v2/public/get_last_trades_by_currency_and_time'
    with requests.Session() as session:
        while True:
            response = session.get(url, params=params)
            response_data = response.json()
            if len(response_data["result"]["trades"]) == 0:
                break
            derivative_list.extend(response_data["result"]["trades"])
            params["start_timestamp"] = response_data["result"]["trades"][-1]["timestamp"] + 1
            if params["start_timestamp"] >= datetime_to_timestamp(dt.combine(end_date, dt.max.time())):
                break
                
    derivative_data = pd.DataFrame(derivative_list)
    if len(derivative_data) == 0:
        return derivative_data
    derivative_data["date_time"] = pd.to_datetime(derivative_data["timestamp"], unit='ms')
    df_duplicated = derivative_data.duplicated(subset=['trade_id'], keep='first')
    derivative_data = derivative_data[~df_duplicated]
    # derivative_data = derivative_data.drop(['block_trade_id'], axis=1)
    derivative_data = derivative_data.assign(strike = derivative_data['instrument_name'].str.extract(r'-(\d+.\d+)-'))
    derivative_data = derivative_data.assign(expire_date = derivative_data['instrument_name'].apply(lambda x : x.split('-')[1]))
    derivative_data = derivative_data.assign(option_type = derivative_data['instrument_name'].apply(lambda x : x.split('-')[3]))
    # derivative_data['expire_date'] = pd.to_datetime(derivative_data['expire_date'], format='%d%b%y')

    # derivative_data['expiry_date'] =derivative_data['expire_date'] - derivative_data['timestamp']
    # derivative_data = derivative_data.sort_values(by=['date_time', 'strike', 'expiry_date', 'option_type'], ascending=[True, True, True, True])


    return derivative_data

In [62]:
# data = derivative_data('BTC', 'option', date(2023, 1, 1), date(2023, 2, 2), 100)

#delete data that data['expire_date'][2:5] is none of  ['MAR', 'JUN', 'SEP', 'DEC']
# data = data[data['expire_date'].str[2:5].isin(['MAR', 'JUN', 'SEP', 'DEC'])]

In [63]:
#iterate all rows and if the low is less than 40
for index, row in iv.iterrows():
    if row['low'] < 35:
        print(row['date'], row['low'])
        day = row['date']
        data = derivative_data('BTC', 'option', date(int(day[:4]), int(day[5:7]), int(day[8:])), date(int(day[:4]), int(day[5:7]), int(day[8:])), 100)
        data = data[data['direction'] == 'buy']
        data = data[data['expire_date'].str[2:5].isin(['MAR', 'JUN', 'SEP', 'DEC'])]
        data.to_csv('data.csv')


2023-07-29 34.11
2023-07-30 34.96
2023-08-04 34.21
2023-08-05 33.42
2023-08-06 33.32
2023-08-07 33.93
2023-08-08 34.76
2023-08-11 32.74
2023-08-12 31.41
2023-08-13 32.18
2023-08-14 32.21
2023-08-15 34.43
2023-08-16 34.23
2023-09-29 33.04
2023-09-30 32.33
2023-10-06 34.47
2023-10-07 34.79
2023-10-13 34.97
2023-10-14 34.82
